In [1]:
!curl ifconfig.co/ip
!curl ifconfig.co/country
!curl ifconfig.co/city

35.187.154.177
Taiwan
Taipei


In [2]:
# 30000142 jita

import concurrent.futures, requests, pytz
import pandas as pd
from datetime import datetime   


# UTC timezone Datetime
dt_local = datetime.now(pytz.utc)
# Convert UTC timezone to Asia/Dhaka timezone
dt_bd = dt_local.astimezone(pytz.timezone('Asia/Dhaka'))
time = dt_bd.strftime("%Y%m%d_%I%M%S%P")



HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
            'Accept-Language': 'en-US;q=0.9'})


URL = f"https://esi.evetech.net/latest/universe/systems/?datasource=tranquility"
res = requests.get(URL, headers=HEADERS)
systems = res.json()
print(f"Total Systems: {len(systems)} \n")


URL = f"https://esi.evetech.net/latest/universe/system_kills/?datasource=tranquility"
res = requests.get(URL, headers=HEADERS)
kills = res.json()
kills_df = pd.DataFrame(kills, index=None)


URL = f"https://esi.evetech.net/latest/universe/system_jumps/?datasource=tranquility"
res = requests.get(URL, headers=HEADERS)
jumps = res.json()
jumps_df = pd.DataFrame(jumps, index=None)


fromJitaDF = pd.read_csv("/content/drive/MyDrive/00_Sheets/EVE_jumps_from_Jita_to_all_systems.csv")

  
def check_info(s):
  system_id = s
  
  URL = f"https://esi.evetech.net/latest/universe/systems/{s}/?datasource=tranquility&language=en"
  res = requests.get(URL, headers=HEADERS)
  system_info = res.json()
  # print(s)
  name = system_info['name']
  security_status = system_info['security_status']
  # print(name, '\n')

  # Sometimes 'security_class' isn't indcluded
  try:
    security_class = system_info['security_class']  
  except:
    security_class = 'NULL'

  # kills_df comes from before
  system_kills = kills_df.loc[kills_df['system_id'] == s]
  if len(system_kills.index):
    npc_kills = system_kills.iloc[0]['npc_kills']
    pod_kills = system_kills.iloc[0]['pod_kills']
    ship_kills = system_kills.iloc[0]['ship_kills']
    # system_id = system_kills.iloc[0]['system_id']
  else:
    npc_kills = 0
    pod_kills = 0
    ship_kills = 0



  # jumps_df comes from before
  system_jump = jumps_df.loc[jumps_df['system_id'] == s]
  if len(system_jump.index):
    ship_jumps = system_jump.iloc[0]['ship_jumps']
  else:
    ship_jumps = 0


  # # jumps form jita (or making csv) #
  # URL = f"https://esi.evetech.net/latest/route/30000142/{s}/?datasource=tranquility&flag=shortest"
  # res = requests.get(URL, headers=HEADERS)
  # from_Jita_shortest = len(res.json())
  # URL = f"https://esi.evetech.net/latest/route/30000142/{s}/?datasource=tranquility&flag=secure"
  # res = requests.get(URL, headers=HEADERS)
  # from_Jita_secure = len(res.json())

  # from CSV data #
  fromJita = fromJitaDF.loc[fromJitaDF['System id'] == s]
  from_Jita_shortest = fromJita.iloc[0]['jump_from_Jita (shortest)']
  from_Jita_secure = fromJita.iloc[0]['jump_from_Jita (secure)']




  dict1 =  {'system_id': system_id,
            'name': name,
            'security_class': security_class,
            'security_status': security_status,
            # 'security_status': round(security_status, 3),
            'ship_jumps': ship_jumps,
            'npc_kills': npc_kills,
            'ship_kills': ship_kills,
            'pod_kills': pod_kills,
            'jump_from_Jita (shortest)': from_Jita_shortest,
            'jump_from_Jita (secure)': from_Jita_secure,
            }

  return dict1


Total Systems: 8435 



In [3]:
%%time
final_df = pd.DataFrame()

with concurrent.futures.ProcessPoolExecutor(max_workers=30) as executor:
  for a in executor.map(check_info, systems):
  # for a in executor.map(check_info, notDone):
    df = pd.DataFrame([a])
    final_df = pd.concat([final_df,df], ignore_index=True)
    # print(a)

CPU times: user 29.7 s, sys: 1.43 s, total: 31.2 s
Wall time: 5min 21s


In [ ]:
print(time)
final_df

In [8]:
print(time)

final_df
# final

20230529_094614pm


,system_id,name,security_class,security_status,ship_jumps,npc_kills,ship_kills,pod_kills,jump_from_Jita (shortest),jump_from_Jita (secure)
0,30000001,Tanoo,B,0.858324,212,97,0,0,20,30
1,30000002,Lashesih,B,0.751689,314,82,0,0,20,30
2,30000003,Akpivem,B,0.846292,174,41,0,0,21,31
3,30000004,Jark,B,0.817001,402,72,0,0,18,28
4,30000005,Sasta,B,0.814337,482,39,1,1,19,29
...,...,...,...,...,...,...,...,...,...,...
8430,34000196,V-196,NULL,-0.990000,0,0,0,0,1,1
8431,34000197,V-197,NULL,-0.990000,0,0,0,0,1,1
8432,34000198,V-198,NULL,-0.990000,0,0,0,0,1,1
8433,34000199,V-199,NULL,-0.990000,0,0,0,0,1,1


In [9]:
# CSV save

final_df.to_csv(f"/content/drive/MyDrive/00_Sheets/eve_security_{time}.csv", encoding = "utf-8",  index=False)

# final.to_csv(f"/content/drive/MyDrive/00_Sheets/EVE__all_systems_names+security+jumpsFromJita.csv", encoding = "utf-8",  index=False)
# final.to_csv(f"/content/drive/MyDrive/00_Sheets/EVE_jumps_from_Jita_to_all_systems.csv", encoding = "utf-8",  index=False)

In [ ]:
done = list(final_df['system_id'])
notDone = [ele for ele in systems if ele not in done]
len(notDone)